## Segment Based Product Recommendation System using Association Rule Learning (ARL) 

##### Link of data: https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

##### In this project I tried to apply Association Rule Learning methods to create a basic recommendation system for retail dataset above

### Imports

In [53]:
import datetime as dt
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from mlxtend.frequent_patterns import apriori, association_rules

##### This is how we read data from your own device
##### There are multiple sheets which are consist of different years
##### In this project we will focus on 2010-2011 sheet

In [62]:
df_ = pd.read_excel("retail.xlsx", sheet_name="Year 2010-2011")

In [135]:
df = df_.copy() # You better take a copy of your data. Because the data is hard to load in case of screw up :)

### Data Preperation

In [134]:
df.shape

(397925, 9)

In [133]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [6]:
df.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,-2.00,1.00,1.00,1.00,3.00,10.00,24.00,29.00,100.0,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,0.19,0.42,0.63,1.25,2.08,4.13,7.95,9.95,18.0,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,12415.00,12626.00,12876.00,13953.00,15152.00,16791.00,17719.00,17905.00,18212.0,18287.0


##### In this data set as can be seen Customer ID and Invoice variables are consist of dublicated records
##### This means a customer can have more than one invoice and an invoice can have more than one items

##### There are some Null values lets get rid of them

In [136]:
df.dropna(axis=0, inplace=True)


##### As mentioned in data source there are some cancelled Invoices in the data set.
##### Cancelled invoices are represented with a capital "C" in the Invoice variable.
##### We need to get rid of them too.

In [137]:
df = df[~df["Invoice"].str.contains("C", na=False)]

##### If we look at varience of Quantity and Price variables we can see that the max values are rediculusly high.
##### Lets pressure them down with the following functions

In [66]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    return up_limit


def replace_with_thresholds(dataframe, variable):
    up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

##### Lets check again

In [11]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397925.0,11.833709,25.534486,1.0,1.00,1.00,1.00,2.00,6.00,12.00,24.00,36.0,120.00,298.50
Price,397925.0,2.893201,3.227143,0.0,0.21,0.42,0.55,1.25,1.95,3.75,6.35,8.5,14.95,37.06
Customer ID,397925.0,15294.308601,1713.172738,12346.0,12415.00,12627.00,12883.00,13969.00,15159.00,16795.00,17725.00,17912.0,18211.00,18287.00


##### Looks great
##### Lets create a variable called "Total Price" by multiplying Price and Quantity variables to calculate the amount paid in total

In [138]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

### Calculating Customer Life Time Values (CLTV)

##### As we are going to make a segment based recommendation system, we need to create segments first.
##### We will create segments according to customers CLTV score wich means we need to calculate CLTV scores for starters
##### To do that we need to have recency, frequency, monetary variables
##### recency: means how many days have passed since customers last purches
##### frequency: means how many purches a customer had until today
##### monetary: means how much money spent by a customer until today
##### T: means how many days have passed since customers first purches (Customer's Age)
##### Lets create them

##### But what is today? We need to define a "today" first :)

In [68]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [153]:
today_date = dt.datetime(2011, 12, 11)

##### We need to take a look at max value in "InvoiceDate" variable and chose a date later than the maximum value 
##### so that we dont get negative values in recency variable we will create in a bit
##### thats why we chose df["InvoiceDate"].max() + 2 days

##### Now we can create rfm table

In [154]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                                    lambda date: (today_date - date.min()).days],
                                                'Invoice': lambda num: num.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ['recency_cltv_p', 'T', 'frequency', 'monetary']

##### Lets see our rfm table loks like

In [155]:
rfm.head()

,recency_cltv_p,T,frequency,monetary
Customer ID,,,,
12346.0,0,326,1,77183.60
12347.0,365,368,7,4310.00
12348.0,282,359,4,1797.24
12349.0,0,19,1,1757.55
12350.0,0,311,1,334.40


In [156]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
recency_cltv_p,4339.0,130.418530,132.039186,0.0,0.000,0.000,0.000,0.000,92.00,251.50,339.000,356.000,367.0000,373.00
T,4339.0,223.782899,117.883623,1.0,10.000,26.000,46.000,113.000,249.00,327.00,367.000,372.000,374.0000,374.00
frequency,4339.0,4.271952,7.705493,1.0,1.000,1.000,1.000,1.000,2.00,5.00,9.000,13.000,30.0000,210.00
monetary,4339.0,2053.797166,8988.247836,0.0,52.076,112.245,156.566,307.245,674.45,1661.64,3646.164,5840.182,19880.0918,280206.02


##### As can be seen there are customers with "0" monetary which is weird because they got invoices but the amount is "0"
##### Lets get rid of them
##### Also to calculate a life time value we better select customers which has more than one purcheses
##### Lets do that too 

In [157]:
rfm = rfm[rfm["monetary"] > 0]

rfm = rfm[(rfm['frequency'] > 1)]

##### Now lets change "monetary" variable into "avg_monetary" by diveding "frequency" variable to calculate avarage monetary value per purches

In [158]:
rfm["monetary"] = rfm["monetary"] / rfm["frequency"]

rfm.rename(columns={"monetary": "monetary_avg"}, inplace=True)

##### Lets check again if it all worked out

In [159]:
rfm.head()

,recency_cltv_p,T,frequency,monetary_avg
Customer ID,,,,
12347.0,365,368,7,615.714286
12348.0,282,359,4,449.310000
12352.0,260,297,8,313.255000
12356.0,302,326,3,937.143333
12358.0,149,151,2,584.030000


In [160]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
recency_cltv_p,2845.0,198.905448,113.866961,0.00,0.0000,16.0000,35.000,98.0000,207.000000,299.000,352.000000,361.800,369.000000,373.00
T,2845.0,258.784183,102.990922,3.00,23.4400,55.0000,85.000,192.0000,283.000000,359.000,369.000000,373.000,374.000000,374.00
frequency,2845.0,5.990158,9.054720,2.00,2.0000,2.0000,2.000,2.0000,4.000000,6.000,11.600000,17.000,37.000000,210.00
monetary_avg,2845.0,422.474779,1658.556123,3.45,56.3585,105.1604,134.333,193.4975,303.716667,441.575,667.472806,929.592,1976.370571,84236.25


##### Everything looks allright

##### To calculate cltv we will use BGNBD Model and Gama Gama Model. In order to use this model we neet to change our "Recency" and "T"(Customer Age) variables into weeks

In [161]:
rfm["recency_weekly_cltv_p"] = rfm["recency_cltv_p"] / 7
rfm["T_weekly"] = rfm["T"] / 7

##### Now we are good to go for building BGNBD Model

In [162]:
bgf = BetaGeoFitter(penalizer_coef=0.01)
bgf.fit(rfm['frequency'],
        rfm['recency_weekly_cltv_p'],
        rfm['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.06, alpha: 9.30, b: 1.13, r: 1.78>

##### With this model we are able to calculate expected purchases of every single customer in the data set. 
##### Also if we sum up all these purchases we can get the expected revenue of the company

In [163]:
# exp_sales_1_month
rfm["exp_sales_1_month"] = bgf.predict(4, # <= The time period can be changed here (remember that our time period is in weeks)
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'])
# exp_sales_3_month
rfm["exp_sales_3_month"] = bgf.predict(12, # <= The time period can be changed here
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'])
#Note: Here we are not calculating how much money will be spent. We are calculating how many purcheses one customer is likely to make

##### Now we will use Gama Gama model to calculate the expected average profit per customer

In [164]:
# expected_average_profit
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(rfm['frequency'], rfm['monetary_avg'])
rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                         rfm['monetary_avg'])

##### By combining bgf(BGNBD Model) and ggf(Gama Gama Model) we can calculate CLTV for different time periods

In [165]:
cltv = ggf.customer_lifetime_value(bgf,
                                   rfm['frequency'],
                                   rfm['recency_weekly_cltv_p'],
                                   rfm['T_weekly'],
                                   rfm['monetary_avg'],
                                   time=6,   # Here is where we decleare the time period(months) that CLTV will be calculated
                                   freq="W", # Here is where we inform our model that in our data the time period is in weeks(w)
                                   discount_rate=0.01) 

rfm["cltv_p"] = cltv

##### Lets check again if it all worked out

In [166]:
rfm.head()

,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p
Customer ID,,,,,,,,,,
12347.0,365,368,7,615.714286,52.142857,52.571429,0.561684,1.678069,632.019673,2211.388315
12348.0,282,359,4,449.310000,40.285714,51.285714,0.364322,1.087951,470.803034,1067.340288
12352.0,260,297,8,313.255000,37.142857,42.428571,0.739160,2.206857,320.724048,1474.515713
12356.0,302,326,3,937.143333,43.142857,46.571429,0.333040,0.993934,996.409652,2062.033986
12358.0,149,151,2,584.030000,21.285714,21.571429,0.474048,1.408955,641.967861,1873.861370


##### Everything looks allright

##### Now in order "cltv_p" score we calculated to make sense to us lets make it scaleable

In [167]:
scaler = MinMaxScaler(feature_range=(1, 100))
scaler.fit(rfm[["cltv_p"]])
rfm["cltv_p"] = scaler.transform(rfm[["cltv_p"]])

##### We can now confidently segment our customers according to their cltv values

In [168]:
rfm["cltv_p_segment"] = pd.qcut(rfm["cltv_p"], 4, labels=["bronze","silver", "gold", "premium"])

##### Here we diveded customers into segments using quartiles but this segmentation can be customized with using more complex structures

##### Now We are succesfully segmented our customers. Lets sort them down and take a look

In [169]:
rfm.sort_values(by="cltv_p", ascending=False).head()

,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,
16446.0,204,206,2,84236.250000,29.142857,29.428571,0.378354,1.126338,92300.885839,100.000000,premium
14646.0,353,355,74,3786.567838,50.428571,50.714286,5.035170,15.049808,3795.579076,55.682751,premium
18102.0,366,368,60,4327.621667,52.285714,52.571429,3.981335,11.900989,4340.321694,50.453728,premium
17450.0,359,368,46,4229.365000,51.285714,52.571429,3.072429,9.183860,4245.570391,38.328453,premium
14096.0,97,102,17,3833.222941,13.857143,14.571429,3.112278,9.255671,3873.246509,35.102345,premium


##### We are done with segmentation 

### Association Rule Learning

In [170]:
rfm.head()

,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,
12347.0,365,368,7,615.714286,52.142857,52.571429,0.561684,1.678069,632.019673,2.014616,premium
12348.0,282,359,4,449.310000,40.285714,51.285714,0.364322,1.087951,470.803034,1.489711,gold
12352.0,260,297,8,313.255000,37.142857,42.428571,0.739160,2.206857,320.724048,1.676528,gold
12356.0,302,326,3,937.143333,43.142857,46.571429,0.333040,0.993934,996.409652,1.946090,premium
12358.0,149,151,2,584.030000,21.285714,21.571429,0.474048,1.408955,641.967861,1.859754,premium


##### First lets select the indexes of customers form different segments

In [171]:
premium_segment_ids = rfm[rfm["cltv_p_segment"] == "premium"].index
gold_segment_ids = rfm[rfm["cltv_p_segment"] == "gold"].index
silver_segment_ids = rfm[rfm["cltv_p_segment"] == "silver"].index
bronze_segment_ids = rfm[rfm["cltv_p_segment"] == "bronze"].index

##### Now we can seperate segments using segment indexes created above
##### We will use "Customer ID" as a variable to do that but it is index at the moment. To do that before seperation we need to reset the index

In [172]:
rfm.head()

,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,
12347.0,365,368,7,615.714286,52.142857,52.571429,0.561684,1.678069,632.019673,2.014616,premium
12348.0,282,359,4,449.310000,40.285714,51.285714,0.364322,1.087951,470.803034,1.489711,gold
12352.0,260,297,8,313.255000,37.142857,42.428571,0.739160,2.206857,320.724048,1.676528,gold
12356.0,302,326,3,937.143333,43.142857,46.571429,0.333040,0.993934,996.409652,1.946090,premium
12358.0,149,151,2,584.030000,21.285714,21.571429,0.474048,1.408955,641.967861,1.859754,premium


In [173]:
rfm = rfm.reset_index()

In [174]:
premium_segment_df = df[df["Customer ID"].isin(premium_segment_ids)]
gold_segment_df = df[df["Customer ID"].isin(gold_segment_ids)]
silver_segment_df = df[df["Customer ID"].isin(silver_segment_ids)]
bronze_segment_df = df[df["Customer ID"].isin(bronze_segment_ids)]

##### To create Association Rules among the products we need a pivot table. In this table, columns should be the product names, index should be the invoice numbers
##### and the content of this table should be ones and zeros to determine which invoice includes which products.
##### As we are going to need this table to create Association Rules for each segment lets define a function to do this

In [57]:
def create_invoice_product_df(dataframe):
    
    return dataframe.groupby(['Invoice', 'StockCode'])['Quantity'].sum().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0) 

# We are filling NA values with zeros wich means that invoice does not contain that product
# we are changing quantity value to (one ore zero). 
# Because we are not interested in quantity of purchesed products. We just need it did purchesed or not

##### Now we are good to go for Creating Association Rules
##### As the following steps include heavy calculations and our data is relatively big, I personally do not advise usage of all data 
##### Especially in an environment like Jupyterlab.
##### We will filter the data and create Association Rules only for German Customers as an example here
##### After that we will create Association Rules for different segments we created above

In [81]:
df_pivot = df[df['Country'] == "Germany"]
df_pivot = create_invoice_product_df(df_pivot) # This is the function we crated above


##### lets see what this table looks like

In [85]:
df_pivot.shape #We have got 457 invoice belongs to German Customers and 1665 different products

(457, 1665)

In [86]:
df_pivot.head()

StockCode,10002,10125,10135,11001,15034,15036,15039,16008,16011,16014,...,90161B,90161C,90161D,90201A,90201B,90201C,90201D,90202D,M,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [87]:
frequent_itemsets = apriori(df_pivot, min_support=0.01, use_colnames=True) 

# This function (apriori) creates support values which means purches rate of an item or item combinations 
# We are using "df_pivot" dataframe we created above
# Also we gave a minimum support value to limit the return. 

##### lets sort the values in frequent_itemsets and take a look

In [93]:
frequent_itemsets.sort_values("support", ascending= False).head(10)

,support,itemsets
538,0.818381,(POST)
189,0.245077,(22326)
1864,0.225383,"(POST, 22326)"
191,0.157549,(22328)
1931,0.150985,"(22328, POST)"
220,0.137856,(22423)
242,0.137856,(22554)
1746,0.131291,"(22328, 22326)"
17,0.126915,(20719)
4457,0.124726,"(22328, POST, 22326)"


##### As you can see items and itemsets are listed according to their purches rate
##### For example "POST" is in invoices in 81% of the purcheses or "22326"is in invoice 24% of the purcheses
##### Are you curious about what those items are? Lets make function return product description

In [98]:
def check_id(stock_code):
    product_name = df[df["StockCode"] == stock_code][["Description"]].values[0].tolist()
    return print(product_name)

In [100]:
check_id("POST") # this is as expected not a product but shipping cost obviously.

['POSTAGE']


In [102]:
check_id(22326) # But this is a real product 

['ROUND SNACK BOXES SET OF4 WOODLAND ']


##### If you wonder what this product look like check this link out
##### https://www.amazon.co.uk/Woodland-Design-Round-Snack-Boxes/dp/B00337093K

##### Now we will create rules with frequent_itemsets that we created above

In [104]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01) # Again we determine a minimum treshold value for support metric

##### Rules are created lets sort them down and take a look

In [106]:
rules.sort_values("lift", ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39036,"(21987, 21989)","(21086, 21988, 21094)",0.010941,0.010941,0.010941,1.0,91.4,0.010821,inf
39041,"(21988, 21086)","(21987, 21989, 21094)",0.010941,0.010941,0.010941,1.0,91.4,0.010821,inf
24745,"(21987, 21989)","(21988, 21086)",0.010941,0.010941,0.010941,1.0,91.4,0.010821,inf
24748,"(21988, 21086)","(21987, 21989)",0.010941,0.010941,0.010941,1.0,91.4,0.010821,inf
24749,"(21989, 21086)","(21987, 21988)",0.010941,0.010941,0.010941,1.0,91.4,0.010821,inf


### What does this table mean?
##### antecedents: first itemset
##### consequents: second itemset
##### antecedent support: first itemset's support(purches rate of first itemset)
##### consequent support: second itemset's support(purches rate of second itemset)
##### support: purches rate of first and the second itemset at the same time
##### confidence: purches possibility of second itemset in case of purchising first itemset
##### lift: represets the increase in the purches possibility of second itemset after purchising first itemset
##### leverage: I do not know exactly what this means
##### conviction: I do not know this too

#### Now we will use "consequents" from the table above to determine a product to recommend for our customers according to their segments
#### But before that we need to create rules for our each segment we seperated earlier
#### As this process is kind of requires repeated steps, lets define a function to speed up things a little

In [108]:
def create_rules(dataframe):

    dataframe = create_invoice_product_df(dataframe)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

    return rules

#### Warning! As following steps consist of heavy calculations it might take a few minutes depending on your computer power


In [179]:
rules_for_premium_segment_df = create_rules(premium_segment_df)
product_for_premium = rules_for_premium_segment_df["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

In [180]:
rules_for_gold_segment_df = create_rules(gold_segment_df)
product_for_gold = rules_for_gold_segment_df["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

In [181]:
rules_for_silver_segment_df = create_rules(silver_segment_df)
product_for_silver = rules_for_silver_segment_df["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

In [182]:
rules_for_bronze_segment_df = create_rules(bronze_segment_df)
product_for_bronze = rules_for_bronze_segment_df["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0]

##### We determined the products lets recommend the products we determined to our customers in France

##### To do that lets create a france table and create a "recommended product" variable in it and 
##### fill it with the products we determined above to recommend 

In [202]:
france_ids = df[df["Country"] == "France"]["Customer ID"].drop_duplicates() # Customer ID's of French Customers selected here

In [184]:
rfm["recommended_product"] = ""  # Here we add an empty variable called "recommended_product" to our rfm table

In [209]:
rfm.head()

,Customer ID,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment,recommended_product
0,12347.0,365,368,7,615.714286,52.142857,52.571429,0.561684,1.678069,632.019673,2.014616,premium,
1,12348.0,282,359,4,449.310000,40.285714,51.285714,0.364322,1.087951,470.803034,1.489711,gold,
2,12352.0,260,297,8,313.255000,37.142857,42.428571,0.739160,2.206857,320.724048,1.676528,gold,
3,12356.0,302,326,3,937.143333,43.142857,46.571429,0.333040,0.993934,996.409652,1.946090,premium,
4,12358.0,149,151,2,584.030000,21.285714,21.571429,0.474048,1.408955,641.967861,1.859754,premium,


In [213]:
rec_products_for_Fr = rfm.loc[rfm["Customer ID"].isin(france_ids)] # This is our customers from France

##### Now we fill it with the products we determined above to recommend 

In [229]:
rec_products_for_Fr.loc[(rec_products_for_Fr["cltv_p_segment"] == "premium"), "recommended_product"] = product_for_premium
rec_products_for_Fr.loc[(rec_products_for_Fr["cltv_p_segment"] == "gold"), "recommended_product"] = product_for_gold
rec_products_for_Fr.loc[(rec_products_for_Fr["cltv_p_segment"] == "silver"), "recommended_product"] = product_for_silver
rec_products_for_Fr.loc[(rec_products_for_Fr["cltv_p_segment"] == "bronze"), "recommended_product"] = product_for_bronze

##### Lets see if it all worked out

In [231]:
rec_products_for_Fr.head(10)

,Customer ID,recency_cltv_p,T,frequency,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment,recommended_product
34,12413.0,270,338,3,252.700000,38.571429,48.285714,0.316926,0.945974,269.642717,1.243716,silver,20719
50,12437.0,329,332,18,275.078333,47.000000,47.428571,1.386641,4.142881,277.983070,2.101827,premium,20685
77,12488.0,65,76,3,432.886667,9.285714,10.857143,0.916732,2.716994,460.971502,2.187025,premium,20685
78,12490.0,324,330,10,541.793000,46.285714,47.142857,0.827380,2.471411,551.805376,2.304331,premium,20685
80,12493.0,31,198,2,216.870000,4.428571,28.285714,0.177844,0.529325,239.665467,1.120778,bronze,20727
81,12494.0,349,366,7,178.014286,49.857143,52.285714,0.562613,1.680817,183.115910,1.294440,silver,20719
87,12508.0,2,30,2,199.135000,0.285714,4.285714,0.997679,2.941940,220.232978,1.611173,gold,20685
95,12523.0,312,332,11,162.706364,44.571429,47.428571,0.890184,2.659169,165.675862,1.421403,silver,20719
101,12535.0,94,187,2,358.175000,13.428571,26.714286,0.361144,1.074580,394.495350,1.403453,silver,20719
102,12536.0,15,60,3,4200.610000,2.142857,8.571429,0.915836,2.711232,4461.677780,12.451908,premium,20685


#### Finally our table of French Customers is ready to share with our marketing department
#### As you can see we determined different products accoridng to our customer segmentation

## Conclusion

#### First we segmented our customers according to their CLTV score.
#### Then we determined the products we will recommend according to segments of our customers all over the world.
#### But this is not ideal. Because, eventhough people from different countries can be in same segment, but their purchase patterns and shopping behaviours can be different.
#### To succesfully recommend products to our customers we need a lot of sub-segments to classify them correctly.
#### To acchive that it is needlees to say we need more and more data